In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import numpy as np

In [2]:
df = pd.read_csv('..\Data\Processed\processed_manga.csv')

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Siddh\AppData\Local\Temp\ipykernel_21208\251128808.py:1: SyntaxWarning: invalid escape sequence '\D'
  df = pd.read_csv('..\Data\Processed\processed_manga.csv')


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 968 entries, 0 to 967
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         968 non-null    int64  
 1   title      968 non-null    object 
 2   synopsis   968 non-null    object 
 3   genres     968 non-null    object 
 4   score      968 non-null    float64
 5   image_url  968 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 45.5+ KB


In [58]:
df[df['id']==3]

,id,title,synopsis,genres,score,image_url,content
2,3,20th Century Boys,"As the 20th century approaches its end, people...",Award_Winning Drama Mystery Sci-Fi,8.93,https://cdn.myanimelist.net/images/manga/5/260...,"As the 20th century approaches its end, people..."


In [22]:
df["content"] = (df["synopsis"]+ " ")*3 + " " + (df["genres"]+ " ")*2

In [23]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['content'])
print(tfidf_matrix.shape)


(968, 11929)


In [24]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print("Cosine similarity matrix shape:", cosine_sim.shape)


Cosine similarity matrix shape: (968, 968)


In [69]:
def get_cbf_recommendations(id, top_n=5):
    # Ensure title exists
    if id not in df['id'].values:
        raise ValueError(f"'{id}' not found in dataset.")
    
    # Get index of given title
    idx = df[df['id'] == id].index[0]
    
    # Pair index with similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort by similarity score (descending), skip the first one (itself)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    
    # Get the indices of recommended manga
    manga_indices = [i[0] for i in sim_scores]
    
    # Return a DataFrame of recommendations
    return df.iloc[manga_indices][['id','title', 'genres','synopsis', 'score','image_url']] if 'score' in df.columns else df.iloc[manga_indices][['title', 'genres']]


In [70]:
get_cbf_recommendations(1)

,id,title,genres,synopsis,score,image_url
673,728,Tetsuwan Atom,Adventure Sci-Fi,"In the distant year 2003, Japan is a technolog...",7.30,https://cdn.myanimelist.net/images/manga/2/186...
961,1047,Saint Seiya: The Lost Canvas - Meiou Shinwa,Action Adventure,This manga tells the story of the previous Hol...,8.27,https://cdn.myanimelist.net/images/manga/5/163...
524,549,The Queen's Knight,Adventure Fantasy Romance,When Yuna Lee's mother moves to Germany to pur...,7.38,https://cdn.myanimelist.net/images/manga/5/263...
865,941,Chaosic Rune,Action Drama Supernatural Ecchi,After being the only survivor in a plane crash...,6.71,https://cdn.myanimelist.net/images/manga/3/183...
511,536,Gensou Suikoden III: Unmei no Keishousha,Action Adventure Fantasy,A long war between the Grasslands and Zexen ha...,7.66,https://cdn.myanimelist.net/images/manga/2/180...


In [47]:


# Save
with open("../model/tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(tfidf, f)

with open("../model/cosine_similarity.pkl", "wb") as f:
    pickle.dump(cosine_sim, f)

df.to_csv("../model/final_manga.csv", index=False)


FileNotFoundError: [Errno 2] No such file or directory: '../model/tfidf_vectorizer.pkl'

In [48]:
ratings_df = pd.DataFrame({
    'user_id': [1, 1, 2, 2, 3],
    'manga_id': [1, 2, 2, 3, 1],
    'rating': [5, 4, 3, 4, 5]
})



In [49]:
user_item_matrix = ratings_df.pivot_table(
    index='user_id',
    columns='manga_id',
    values='rating'
).fillna(0)


In [50]:
print(user_item_matrix)

manga_id    1    2    3
user_id                
1         5.0  4.0  0.0
2         0.0  3.0  4.0
3         5.0  0.0  0.0


In [51]:
import numpy as np
from scipy.sparse.linalg import svds

matrix = user_item_matrix.values

# Compute SVD
U, sigma, Vt = svds(matrix, k=2)
sigma = np.diag(sigma)

# Predicted ratings
predicted_ratings = np.dot(np.dot(U, sigma), Vt)
pred_df = pd.DataFrame(predicted_ratings, columns=user_item_matrix.columns, index=user_item_matrix.index)


In [79]:
def get_cf_recommendations(user_id, top_n=5):
    user_preds = pred_df.loc[user_id].sort_values(ascending=False)
    rated = ratings_df[ratings_df.user_id == user_id]['manga_id'].tolist()
    recommended = user_preds.drop(rated).head(top_n)
    return df[df['id'].isin(recommended.index)][['id', 'title','genres','synopsis','score','image_url']]


In [80]:
print(get_cf_recommendations(1))


   id              title                              genres  \
2   3  20th Century Boys  Award_Winning Drama Mystery Sci-Fi   

                                            synopsis  score  \
2  As the 20th century approaches its end, people...   8.93   

                                           image_url  
2  https://cdn.myanimelist.net/images/manga/5/260...  


In [81]:
def hybrid_recommendations(user_id, top_n=5, alpha=0.7):
    """alpha: weight given to CF (0.0 = only CBF, 1.0 = only CF)"""
    
    try:
        cf_recs = get_cf_recommendations(user_id, top_n=top_n)
    except KeyError:
        return get_cbf_recommendations(1, top_n)  # fallback to CBF
    
    cbf_recs = get_cbf_recommendations(1, top_n)  # base this on a manga user liked
    #print(cf_recs,cbf_recs)
    combined = pd.concat([cf_recs, cbf_recs]) 
    #print(combined)
    combined = combined.drop_duplicates('id') \
                 .head(top_n)
    return combined


In [82]:
hybrid_recommendations(1)

,id,title,genres,synopsis,score,image_url
2,3,20th Century Boys,Award_Winning Drama Mystery Sci-Fi,"As the 20th century approaches its end, people...",8.93,https://cdn.myanimelist.net/images/manga/5/260...
673,728,Tetsuwan Atom,Adventure Sci-Fi,"In the distant year 2003, Japan is a technolog...",7.30,https://cdn.myanimelist.net/images/manga/2/186...
961,1047,Saint Seiya: The Lost Canvas - Meiou Shinwa,Action Adventure,This manga tells the story of the previous Hol...,8.27,https://cdn.myanimelist.net/images/manga/5/163...
524,549,The Queen's Knight,Adventure Fantasy Romance,When Yuna Lee's mother moves to Germany to pur...,7.38,https://cdn.myanimelist.net/images/manga/5/263...
865,941,Chaosic Rune,Action Drama Supernatural Ecchi,After being the only survivor in a plane crash...,6.71,https://cdn.myanimelist.net/images/manga/3/183...


In [83]:
import joblib

# Save CBF similarity matrix
joblib.dump(cosine_sim, "../models/cbf_cosine_matrix.pkl")
joblib.dump(tfidf, "../models/tfidf_vectorizer.pkl")

# Save CF SVD components
joblib.dump(U, "../models/cf_U.pkl")
joblib.dump(sigma, "../models/cf_sigma.pkl")
joblib.dump(Vt, "../models/cf_Vt.pkl")


['../models/cf_Vt.pkl']